In [1]:
import numpy as np
import pandas as pd
import math
import time
import pickle as pkl
import gensim
import shutil
import glob
import os
import smart_open
import time

In [54]:
IV_file = '../../data/pickle/inverted_index.pkl'
TAGDICFILE = '../../data/pickle/trainTagsToIndex.pkl'
DOC_LENGTH_FILE = '../../data/pickle/doc_length.pkl'
QUERIES_FILE = '../../data/queries.txt'
DOC2VECMODEL = '../../data/model_embeddings/modelembedding60_40.pkl'
stopwords = open('../code/stopwords.txt', 'r').readlines()
punctuation = open('../code/punctuation.txt', 'r').readlines()
stopwords = [i.strip() for i in stopwords]
punctuation = [i.strip() for i in punctuation]
f = open(IV_file, 'rb')
inverted_index = pkl.load(f)
f.close()
f = open(DOC_LENGTH_FILE, 'rb')
documentLenArr = pkl.load(f)
f.close()
f = open(TAGDICFILE, 'rb')
tagDic = pkl.load(f)
f.close()
numdocuments = len(documentLenArr)
k1 = 1.5
b = 0.5
avgDoclength = sum(documentLenArr)*1.0/numdocuments
maxranking = 50
f = open(DOC2VECMODEL, 'rb')
doc2vecmodel = pkl.load(f)
f.close()


In [3]:
def read_corpus_query(fname):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
#             print(i,line)
            tokens = gensim.utils.simple_preprocess(line)
            yield tokens
            


In [4]:
# assuming numdocuments, documentLenArr, k1, b, avgDoclength, maxrank
#usage sort_index(getScoreForQuery(find_count(query)))
def getScoreForQuery(queryMap):
    documentToScore = {}
    for term, termfreq in queryMap.items():
        if term in inverted_index:
            posting = inverted_index[term]
            weightOrIdf = calcIdf(len(posting))
            for docInfo in posting:
                doc_id = docInfo[0]
                doc_id_freq = docInfo[1]
                scoreTermDoc = getScoreForTermForDocument(termfreq,doc_id,weightOrIdf,doc_id_freq)
                if doc_id in documentToScore:
                    documentToScore[doc_id] = documentToScore[doc_id] + scoreTermDoc
                else:
                    documentToScore[doc_id] = scoreTermDoc
    return documentToScore
def getScoreForQueryOptimized(queryMap, qindex):
    documentToScore = {}
    thisdocset = docsets[qindex]
    for term, termfreq in queryMap.items():
        if term in inverted_index:
            posting = inverted_index[term]
            weightOrIdf = calcIdf(len(posting))
            for docInfo in posting:
                doc_id = docInfo[0]
                if doc_id not in thisdocset:
                    continue
                doc_id_freq = docInfo[1]
                scoreTermDoc = getScoreForTermForDocument(termfreq,doc_id,weightOrIdf,doc_id_freq)
                if doc_id in documentToScore:
                    documentToScore[doc_id] = documentToScore[doc_id] + scoreTermDoc
                else:
                    documentToScore[doc_id] = scoreTermDoc
    return documentToScore
def calcIdf(lenposting):
    ans = math.log((numdocuments*1.0 - lenposting +0.5)/(lenposting+0.5))
    return ans
def getScoreForTermForDocument(qfi,docNo,weight,tfi):
    docLength = documentLenArr[docNo]*1.0
    ans = weight* ((tfi*(k1+1)*1.0)/(tfi+k1*(1.0-b+b*(docLength/avgDoclength))))
    ans = ans*qfi
    return ans
def find_count(text_arr):
    dictionary = {}
    for i in text_arr:
        if (i not in stopwords and i not in punctuation):
            if (i in dictionary):
                dictionary[i] += 1
            else:
                dictionary[i] = 1
    return dictionary

In [5]:
def sort_index(documentToScore):
    ans = []
    numvaluesAdded=0
    while(numvaluesAdded<=maxranking and len(documentToScore)!=0):
        tempkey = max(documentToScore, key=documentToScore.get)
        ans.append((tempkey, documentToScore[tempkey]))
        numvaluesAdded +=1
        del documentToScore[tempkey]
    return ans
def tagToScore(documentToScore):
    ans = []
    for doc_id, score in documentToScore:
        ans.append((tagDic[doc_id], score))
    return ans;

In [6]:
def writeOutput(answers, filename):
    ansstr = ""
    for i in range(len(answers)):
        thisanswer = answers[i]
        doctagToScore = tagToScore(thisanswer)
        for j in range(len(doctagToScore)):
            tempstr = str(i+51) + " 0 "+ doctagToScore[j][0] + " " + str(j+1)+" "+ str(doctagToScore[j][1])+ " p\n"
#             print(tempstr)
            ansstr += tempstr
    outfile = open(filename, "w")
    outfile.write(ansstr)
    outfile.close()
    

In [ ]:
writeOutput(answersall, "../../data/answers.txt")

In [ ]:
writeOutput(answersallOptimized, "../../data/answersOptimized.txt")

In [ ]:
%%time
answers0 = sort_index(getScoreForQuery(find_count(queries[0].split(' '))))

In [ ]:
%%time

answers0opti = sort_index(getScoreForQueryOptimized(find_count(queries[0].split(' ')),0))

In [ ]:
answers0

In [ ]:
answers0opti

In [7]:
%%time
starttime = time.time()
queries = open(QUERIES_FILE, 'r').readlines()
answersall = []
for x in range(len(queries)):
    answersall.append(sort_index(getScoreForQuery(find_count(queries[x].split(' ')))))
endtime = time.time()

CPU times: user 1min 31s, sys: 7.72 ms, total: 1min 31s
Wall time: 1min 31s


In [8]:
print(endtime-starttime)

91.15344500541687


In [44]:
def minnextDoc(leastdoc):
    mindoc = numdocuments
    minkey = None
    for key,value in nextDocDic.items():
#         if key not in inverted_index:
#             continue
        keyInv = inverted_index[key]
        if(value+1<=len(keyInv)-1):
            thisdoc = inverted_index[key][value+1][0]
            if(thisdoc>leastdoc and thisdoc<mindoc):
                mindoc = thisdoc
                minkey = key
    if(minkey!=None):
        nextDocDic[minkey] +=1
        return mindoc
    else:
        return numdocuments
def minnextDocOptimized(leastdoc, qindex):
    mindoc = numdocuments
    minkey = None
    for key,value in nextDocDic.items():
#         if key not in inverted_index:
#             continue
        keyInv = inverted_index[key]
        if(value+1<=len(keyInv)-1):
            thisdoc = inverted_index[key][value+1][0]
            if(thisdoc>leastdoc and thisdoc<mindoc):
                if(thisdoc in docsets[qindex]):
                    mindoc = thisdoc
                    minkey = key
    if(minkey!=None):
        nextDocDic[minkey] +=1
        return mindoc
    else:
        return numdocuments
def getDAATScore(docno):
    documentToScore = {}
    docScore = 0 
    for term, termfreq in queryMap.items():
        if term in inverted_index:
            posting = inverted_index[term]
            weightOrIdf = calcIdf(len(posting))
            docNolist = posting[:,0]
            findex = np.searchsorted(docNolist, docno)
#             docInfo = next((x for x in posting if x[0]==docno), None)
            
#             print(docInfo)
#             print(type(docInfo))
#             if docInfo is not None:
            if (findex!=len(posting)):
                docInfo = posting[findex]
                doc_id = docInfo[0]
                doc_id_freq = docInfo[1]
                scoreTermDoc = getScoreForTermForDocument(termfreq,doc_id,weightOrIdf,doc_id_freq)
                docScore += scoreTermDoc
    return docScore

In [40]:
%%time
answersall = []
queries = open(QUERIES_FILE, 'r').readlines()[0:2]
for x in range(2):
    print("query x", x)
    results={}
    r=0
    queryMap = find_count(queries[x].split(' '))
    nextDocDic = {}
    for key in queryMap:
        if key in inverted_index:
            nextDocDic[key] = -1
    d = minnextDoc(-1)
    while (d!=numdocuments):
#         print("document", d)
        scoreford = getDAATScore(d)
    #     results.append((d,scoreford))
        results[d] = scoreford
        d = minnextDoc(d)
#     results = sort_index(results)
    answersall.append(sort_index(results))
    

query x 0
query x 1
CPU times: user 32.4 s, sys: 303 ms, total: 32.7 s
Wall time: 32.4 s


In [59]:
%%time
starttime = time.time()
queries = open(QUERIES_FILE, 'r').readlines()
test_corpus = list(read_corpus_query(QUERIES_FILE))
docsets = []
for doc_id in range(len(test_corpus)):
    inferred_vector = doc2vecmodel.infer_vector(test_corpus[doc_id])
    sims = doc2vecmodel.docvecs.most_similar([inferred_vector], topn=80000)
#     print(sims[:])
    docsets.append(set([x for x,_ in sims]))

CPU times: user 12.8 s, sys: 135 ms, total: 13 s
Wall time: 13.3 s


In [60]:
%%time
answersallOptimized = []
queries = open(QUERIES_FILE, 'r').readlines()
for x in range(len(queries)):
    print("query x", x)
    results={}
    r=0
    queryMap = find_count(queries[x].split(' '))
    nextDocDic = {}
    for key in queryMap:
        if key in inverted_index:
            nextDocDic[key] = -1
    d = minnextDocOptimized(-1,x)
    while (d!=numdocuments):
#         print("document", d)
        scoreford = getDAATScore(d)
    #     results.append((d,scoreford))
        results[d] = scoreford
        d = minnextDocOptimized(d,x)
#     results = sort_index(results)
    answersallOptimized.append(sort_index(results))
    

query x 0
query x 1
query x 2
query x 3
query x 4
query x 5
query x 6
query x 7
query x 8
query x 9
query x 10
query x 11
query x 12
query x 13
query x 14
query x 15
query x 16
query x 17
query x 18
query x 19
query x 20
query x 21
query x 22
query x 23
query x 24
query x 25
query x 26
query x 27
query x 28
query x 29
query x 30
query x 31
query x 32
query x 33
query x 34
query x 35
query x 36
query x 37
query x 38
query x 39
query x 40
query x 41
query x 42
query x 43
query x 44
query x 45
query x 46
query x 47
query x 48
query x 49
CPU times: user 52.1 s, sys: 2.66 s, total: 54.8 s
Wall time: 52.5 s


In [41]:
writeOutput(answersall, "../../data/answersdaat.txt")

In [46]:
writeOutput(answersallOptimized, "../../data/answersOptimizeddaat.txt")

In [53]:
answersall

[[(36803, 642.0171646474662),
  (36812, 639.6752536603939),
  (36800, 638.6834493300521),
  (36906, 637.3198187270344),
  (36905, 637.0341105570151),
  (36838, 636.0100838244969),
  (1953, 635.7081678801397),
  (54284, 635.3409064715265),
  (36926, 634.838768653374),
  (36903, 633.4091400684983),
  (54270, 631.6473144943026),
  (36786, 631.481167794616),
  (53415, 630.5894989054217),
  (52057, 630.4914581358734),
  (42250, 630.2790835245827),
  (36920, 630.0166684731372),
  (36923, 629.789609753498),
  (42255, 629.1323334117371),
  (36859, 628.1360653802095),
  (51884, 627.900485126299),
  (36961, 627.3295824207974),
  (52151, 626.7049447102514),
  (51883, 626.2382871000215),
  (53531, 625.9945607309712),
  (51886, 625.65882485759),
  (55318, 625.583021999232),
  (55320, 625.5667375961688),
  (36789, 625.2273378714503),
  (23300, 625.1623086130013),
  (55325, 624.9833334836458),
  (36959, 624.9564424853531),
  (36714, 624.5036028584399),
  (54299, 624.2185521684282),
  (36971, 624.2087

In [61]:
answersallOptimized

[[(1953, 635.7081678801397),
  (36977, 623.1947724797975),
  (4555, 615.4707152777063),
  (12474, 608.1122018882529),
  (10346, 604.1756807707843),
  (3777, 602.7417860032502),
  (10344, 602.3252589479258),
  (4309, 601.0810075895599),
  (10333, 599.4622190564335),
  (3903, 597.3233203990873),
  (4110, 597.238722825067),
  (3683, 596.7652752048798),
  (3812, 594.8043721353316),
  (10336, 594.3574223175441),
  (10176, 593.9612222761137),
  (10413, 593.7876782521282),
  (3029, 592.3057121486088),
  (10309, 591.8072346225814),
  (74614, 589.9746658727694),
  (26981, 589.488949673418),
  (4200, 589.2005955352887),
  (10584, 587.6316646271704),
  (3977, 587.1795490827346),
  (10071, 587.0148191286821),
  (68360, 586.900566115003),
  (3894, 586.3410935741916),
  (7171, 585.7603857350549),
  (3917, 585.2819036041126),
  (7052, 584.9717292634097),
  (7617, 584.9424533091494),
  (6652, 584.1014115753915),
  (55143, 583.970484321925),
  (4265, 583.901981032747),
  (3949, 583.5999201204772),
  (4

In [36]:
sort_index(results)

[(36803, 642.0171646474662),
 (36812, 639.6752536603939),
 (36800, 638.6834493300521),
 (36906, 637.3198187270344),
 (36905, 637.0341105570151),
 (36838, 636.0100838244969),
 (1953, 635.7081678801397),
 (54284, 635.3409064715265),
 (36926, 634.838768653374),
 (36903, 633.4091400684983),
 (54270, 631.6473144943026),
 (36786, 631.481167794616),
 (53415, 630.5894989054217),
 (52057, 630.4914581358734),
 (42250, 630.2790835245827),
 (36920, 630.0166684731372),
 (36923, 629.789609753498),
 (42255, 629.1323334117371),
 (36859, 628.1360653802095),
 (51884, 627.900485126299),
 (36961, 627.3295824207974),
 (52151, 626.7049447102514),
 (51883, 626.2382871000215),
 (53531, 625.9945607309712),
 (51886, 625.65882485759),
 (55318, 625.583021999232),
 (55320, 625.5667375961688),
 (36789, 625.2273378714503),
 (23300, 625.1623086130013),
 (55325, 624.9833334836458),
 (36959, 624.9564424853531),
 (36714, 624.5036028584399),
 (54299, 624.2185521684282),
 (36971, 624.2087100926894),
 (53408, 623.880085277

In [ ]:
tagToScore(answers0)

In [ ]:
%%time
starttime = time.time()
queries = open(QUERIES_FILE, 'r').readlines()
test_corpus = list(read_corpus_query(QUERIES_FILE))
docsets = []
for doc_id in range(len(test_corpus)):
    inferred_vector = doc2vecmodel.infer_vector(test_corpus[doc_id])
    sims = doc2vecmodel.docvecs.most_similar([inferred_vector], topn=10000)
#     print(sims[:])
    docsets.append(set([x for x,_ in sims]))
answersallOptimized = []
for x in range(len(queries)):
    answersallOptimized.append(sort_index(getScoreForQueryOptimized(find_count(queries[x].split(' ')),x)))
endtime = time.time()

In [ ]:
print(endtime-starttime)

In [ ]:
tagToScore(answersallOptimized[0])

In [ ]:
avgDoclength

In [ ]:
26981 in allsims[0]

In [ ]:
[(x[0] in allsims[0]) for x in answers0]